# Intro

This notebook applies the HSV colour based segmentation process to create masks of individual embryos

The outputs of this notebook and [1a_gradient_segmentation](1a_gradient_segmentation.ipynb) are combined (in downstream processes) to generate the final segmentation mask

# Imports

In [1]:
import os
import shutil
import numpy as np
import cv2
from joblib import delayed, Parallel
from tqdm import tqdm

from fam13a import utils, image
from fam13a.image.consts import HSV_MIN_THRESHOLD

# Constants

In [6]:
PROJ_ROOT = utils.here()
DATA_ROOT = os.path.join(PROJ_ROOT, 'data', 'interim', 'xenopus')
OUTPUT_ROOT = os.path.join(PROJ_ROOT, 'data', 'processed', 'xenopus', 'segmented', 'colour')
NCPUS = 8

# define the cropping to apply to all frames in a video
# use None to signify no cropping as slice(0, 0) will pick out an empty array
CROPS = (slice(50, None), slice(None, None))

HSV_MAX_THRESHOLD = np.array([255, 255, 255])

file_ids = [_f.split('.mp4')[0] for _f in os.listdir(DATA_ROOT) if _f.endswith('mp4')]
print(file_ids)

['20_L2_MO_late_3', '10_L2_3', '15_L2_MO_late_1', '5_L2_3', 'C_MO_2', '20_L2_MO_late_2', 'C_MO_3', '5_L2_2', '10_L2_2', '15_L2_MO_late_3', '5_L2_1', '10_L2_1', '20_L2_MO_late_1', '15_L2_MO_late_2', 'C_MO_1']


# Setup

In [ ]:
# define a simple wrapper function for the segmentation to make it easy to parallelise
def process(frame, label, min_thr, max_thr, output, zfill_val):
    # mask each frame based on whether the pixel values are in the 
    # given range in HSV colour space 
    mask = cv2.inRange(frame, min_thr, max_thr)
    name = f'{label}'.zfill(zfill_val)
    np.save(os.path.join(output, name), mask)

# Processing

Each video takes ~20s to process on 8 CPUs

In [ ]:
for file_count, file_id in enumerate(file_ids):
    print(f'Started processing: {file_id} ({file_count+1} of {len(file_ids)})')
    # ensure output directory exists and it is empty
    output = os.path.join(OUTPUT_ROOT, file_id)
    try:
        shutil.rmtree(output)
    except FileNotFoundError:
        pass
    os.makedirs(output)
    
    # load video frames
    frames = utils.frames_from_video(os.path.join(DATA_ROOT, f'{file_id}.mp4'))
    # calculate the zfill value to use on the output file names so they are 
    # sorted alphanumerically by name
    zfill_val = len(str(frames.shape[0]))
    # convert frames from BGR to HSV colour format for segmentation
    frames = [cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) for frame in frames]

    # crop all frames in the video as specified
    frames = np.stack(frames, axis=0)[(..., *CROPS)]

    with Parallel(n_jobs=NCPUS, verbose=1) as par:
        par(delayed(process)(
            frame, label, HSV_MIN_THRESHOLD, HSV_MAX_THRESHOLD, output, zfill_val
        ) for label, frame in enumerate(frames))